In [171]:
import open3d as o3d
import numpy as np

# Load the raw point cloud
MM_TO_M = 1 / 1000.0
pcd_path = "PointCloud/PointCloud_box_a2.pcd" # only a3 has problem due to sparse top
pcd = o3d.io.read_point_cloud(pcd_path)

# Convert units (assumes points are in mm)
points = np.asarray(pcd.points, dtype=np.float64) * MM_TO_M
pcd.points = o3d.utility.Vector3dVector(points)

# Visualize raw point cloud
o3d.visualization.draw_geometries([pcd], window_name="Raw Point Cloud")

### Fixed tiled Camera - Not actually required?

tilt adjustment is automatic, but must follow these conditions: 

- The table must be the largest flat surface in the scene.

- The point cloud must include enough of the table — not just the box.

- Camera must not be too far tilted (e.g., looking from the side).

If those conditions hold (which they usually do in top-down LIDAR setups), then RANSAC will find the table plane accurately, and the leveling will succeed.

https://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html

In [104]:
# Adjust point cloud to level -> adjust tilt with RANSAC
# Rotate the point cloud so the dominant plane (which is table) becomes horizontal
def level_cloud(pcd, distance_thresh=0.002, num_iters=1000): # set RANSAC distance_thresh and num_iters
    # Use RANSAC (random sample consensus (RSC)) to segment the dominant plane: ax + by + cz + d = 0, 3 main steps in RANSAC: 
    # 1. Randomly sample 3 points from point cloud
    # 2. Estimate a plane from the 3 points: ax + by + cz + d = 0
    # 3. Retain points that are close to the estimated plane
    (a, b, c, d), inliers = pcd.segment_plane(distance_thresh,
                                              ransac_n=3,
                                              num_iterations=num_iters)
    
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    
    # Get and normalize the normal vector of the plane
    n = np.array([a, b, c])
    n /= np.linalg.norm(n)

    # Target vector (we want to rotate n to align with Z-axis)
    z = np.array([0, 0, 1])

    # Compute rotation axis (cross product between n and Z)
    v = np.cross(n, z)
    s = np.linalg.norm(v)

    # Compute rotation matrix
    if s < 1e-6:
        # If n is already aligned with Z, no rotation needed
        R = np.eye(3)
    else:
        # # Compute rotation matrix using Rodrigues rotation formula
        c_dot = np.dot(n, z)
        vx = np.array([[0, -v[2], v[1]],
                       [v[2], 0, -v[0]],
                       [-v[1], v[0], 0]])
        R = np.eye(3) + vx + vx @ vx * ((1 - c_dot) / (s ** 2))

    centroid = np.asarray(pcd.points)[inliers[0]]

    # Translate to origin -> rotate -> translate back
    pcd = pcd.translate(-centroid, relative=False)
    pcd = pcd.rotate(R, center=(0, 0, 0))
    pcd = pcd.translate(centroid, relative=False)
    return pcd

pcd = level_cloud(pcd)
o3d.visualization.draw_geometries([pcd], window_name="Leveled Point Cloud")


Plane equation: -0.01x + -0.13y + 0.99z + -0.64 = 0


In [101]:
# Save PCD
pcd.points = o3d.utility.Vector3dVector(
    np.asarray(pcd.points) * 1000.0) # convert back m → mm

o3d.io.write_point_cloud("tilted_mm_ascii.pcd",
                         pcd,
                         write_ascii=True)

True

### Remove noise (optional step) - Not requried to work

Might actually be worse to have this, since it remove some points on box

In [102]:
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
o3d.visualization.draw_geometries([pcd], window_name="After Noise Removal")

### Fit and Extract the Table Plane

In [172]:
# Fit a plane to the point cloud using RANSAC, similar to level_cloud using RANSAC
def plane_fit(cloud, thresh=0.005, n_iter=1000):
    # Fit with RANSAC
    (a, b, c, d), inliers = cloud.segment_plane(thresh, ransac_n=3, num_iterations=n_iter)
     # Normalize the normal vector and d value
    n = np.array([a, b, c])
    norm = np.linalg.norm(n)
    n /= norm
    d /= norm
    return n, d, inliers

n_tab, d_tab, table_inliers = plane_fit(pcd) # Finds the table plane, extract normal vecotr (n_tab) and offset (d_tab)
table_pc = pcd.select_by_index(table_inliers) # table object (dominant flat plane)
obj_pc = pcd.select_by_index(table_inliers, invert=True) # all other object, found by using invert=True

# Visualize
table_pc.paint_uniform_color([1, 0, 0])
obj_pc.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([table_pc, obj_pc], window_name="Table (Red) and Object (Gray)")


In [173]:
o3d.visualization.draw_geometries([obj_pc]) # visualize just one item

### Cluster Remaining Points to Identify the Box

Identifying and isolating the box (or object) from the set of non-table points using clustering
- eps = 0.02: max distance (in meter) between neighboring points in a cluster (2 cm)
- min_points = 20: minimum number of points to form a dense cluster

Before this, let's apply a Z-threshold filter based on the detected table plane height, before clustering.

In [174]:
# cluster non-table points that are above the table surface, z-filter step
# Convert to NumPy array
obj_points = np.asarray(obj_pc.points)

# Compute Z of table at origin (assuming table normal is upward)
# Plane eq: ax + by + cz + d = 0 → at (0, 0, z): cz + d = 0 → z = -d/c
table_z = -d_tab / n_tab[2]  # Estimate table height at Z-axis intercept

# Add small offset to avoid points just slightly below the surface
z_margin = 0.005  # 5 mm
mask = obj_points[:, 2] <= (table_z + z_margin)

# Filter only points above the table
obj_above_table = obj_pc.select_by_index(np.where(mask)[0])

o3d.visualization.draw_geometries([obj_above_table]) # visualize just one item

In [175]:
labels = np.array(obj_above_table.cluster_dbscan(eps=0.02, min_points=20)) # DBSCAN clustering
largest = np.bincount(labels[labels >= 0]).argmax() # Finds the most populated cluster (assumed to be the box)
box_pc = obj_above_table.select_by_index(np.where(labels == largest)[0]) # Selects only the points that belong to the largest cluster.

# Visualize
box_pc.paint_uniform_color([0.2, 0.8, 1.0])
o3d.visualization.draw_geometries([box_pc], window_name="Detected Box Cluster")

### Fit the Lid Plane of the Box

In [180]:
# RANSAC to find dominant plane, this will be top of box
n_lid, d_lid, lid_inliers = plane_fit(box_pc)
lid_pc = box_pc.select_by_index(lid_inliers)

print(f"Plane: {n_lid} * x + {d_lid:.2f} = 0")

Plane: [-0.06369408 -0.16519766  0.9842016 ] * x + -0.55 = 0


In [181]:
lid_pc.paint_uniform_color([0.0, 1.0, 0.0])
box_pc.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([lid_pc, box_pc], window_name="Lid (Green) on Box")

### Measure Dimensions (L, W, H)

In [185]:
# Height from plane offset difference
height_m = abs(d_lid - d_tab)

# XY from lid's oriented bounding box
lid_obb = lid_pc.get_oriented_bounding_box()
xy_m = lid_obb.extent[:2]

# Final dimensions in cm
dims_cm = np.array([xy_m[0], xy_m[1], height_m]) * 100.0
print(f"Box ≈ {dims_cm[0]:.2f} × {dims_cm[1]:.2f} × {dims_cm[2]:.2f} cm  (L×W×H)")

Box ≈ 20.84 × 11.12 × 8.70 cm  (L×W×H)


### Visualize Final Result

In [186]:
lid_obb.color = (1.0, 0.5, 0.0)  # bounding box outline = orange
table_pc.paint_uniform_color([1.0, 0.0, 0.0]) # table = red
lid_pc.paint_uniform_color([0.0, 1.0, 0.0]) # lid = green
box_pc.paint_uniform_color([0.2, 0.8, 1.0]) # box body = cyan

o3d.visualization.draw_geometries(
    [table_pc, lid_pc, box_pc, lid_obb],
    window_name="Final Visualization: Table, Lid, Box, OBB",
    width=800, height=600
)

### Test with more bounding box

In [157]:
# Create more bounding boxes
table_obj = table_pc.get_oriented_bounding_box()

In [158]:
lid_obb.color = (1.0, 1.0, 0.0)  # yellow box
table_obj.color = (1.0, 0.0, 0.0)
lid_pc.paint_uniform_color([0.0, 1.0, 0.0])
box_pc.paint_uniform_color([0.2, 0.8, 1.0])

o3d.visualization.draw_geometries(
    [table_obj, table_pc, lid_pc, box_pc, lid_obb],
    window_name="Final Visualization: Table, Lid, Box, OBB",
    width=800, height=600
)